# Cloud Storage Extension Configuration

This notebook configures [cloud-storage-extension](https://github.com/exasol/cloud-storage-extension/) using 
functions from [notebook-connector library](https://github.com/exasol/notebook-connector/).

On the high level it does the following:
* retrieves latest release of cloud-storage-extension jar
* uploads it into [BucketFS](https://docs.exasol.com/db/latest/database_concepts/bucketfs/bucketfs.htm)
* registers UDF scripts for the cloud-storage-extension

## Cloud Storage Extension setup

In [ ]:
from exasol.nb_connector import github, bfs_utils, cloud_storage
from exasol.nb_connector.connections import open_bucketfs_connection, open_pyexasol_connection
import exasol.bucketfs as bfs
import pathlib

jar_local_path = github.retrieve_jar(github.Project.CLOUD_STORAGE_EXTENSION, use_local_cache=True)
local_name = jar_local_path.name
bfs_path = bfs.path.BucketPath(local_name, open_bucketfs_connection(ai_lab_config))
with jar_local_path.open("rb") as file:
    bfs_path.write(file)
udf_path = bfs_path.as_udf_path()

with open_pyexasol_connection(ai_lab_config) as conn:
    cloud_storage.setup_scripts(conn, ai_lab_config.db_schema, udf_path)
    
print("Could Storage Extension was initialized")